## Homework 11:
Self build neural network for GUI

In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import fashion_mnist
import numpy as np
import matplotlib.pyplot as plt
import imageio as io
import cv2

# !pip install tensorflow-model-optimization tf-keras
import tensorflow_model_optimization as tfmot
import tempfile
import os
import zipfile

In [4]:
# Load the Fashion MNIST dataset
fashion_mnist = tf.keras.datasets.fashion_mnist
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Normalize the data
x_train = x_train / 255.0
x_test = x_test / 255.0

# Reshape for CNN input
x_train = x_train[..., np.newaxis]
x_test = x_test[..., np.newaxis]


In [3]:
# Define the model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')  # 10 classes
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1600)              0         
                                                                 
 dense (Dense)               (None, 128)               2

In [7]:
# Train the model
model.fit(x_train, y_train, epochs=5, batch_size=64, validation_data=(x_test, y_test))

Epoch 1/5
938/938 [==============================] - 60s 63ms/step - loss: 0.5244 - accuracy: 0.8100 - val_loss: 0.3823 - val_accuracy: 0.8636
Epoch 2/5
938/938 [==============================] - 48s 51ms/step - loss: 0.3389 - accuracy: 0.8767 - val_loss: 0.3240 - val_accuracy: 0.8809
Epoch 3/5
938/938 [==============================] - 40s 43ms/step - loss: 0.2899 - accuracy: 0.8943 - val_loss: 0.3137 - val_accuracy: 0.8841
Epoch 4/5
938/938 [==============================] - 48s 51ms/step - loss: 0.2589 - accuracy: 0.9053 - val_loss: 0.2877 - val_accuracy: 0.8917
Epoch 5/5
938/938 [==============================] - 44s 47ms/step - loss: 0.2371 - accuracy: 0.9129 - val_loss: 0.2775 - val_accuracy: 0.8994


In [8]:
# save model
model.save("model_homework11.h5")

In [9]:
# load model from h5 file
model = tf.keras.models.load_model("model_homework11.h5")

In [10]:
model.evaluate(x_test, y_test)

313/313 [==============================] - 3s 8ms/step - loss: 0.2775 - accuracy: 0.8994


[0.27753204107284546, 0.899399995803833]

### Save as tf lite model with quantization

In [11]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS,
                                       tf.lite.OpsSet.SELECT_TF_OPS]

def representative_dataset():
    for i in range(200):
        yield [np.asarray(x_train[i], dtype='float32')[None, ...]]

converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS,
                                       tf.lite.OpsSet.TFLITE_BUILTINS_INT8,
                                       tf.lite.OpsSet.SELECT_TF_OPS]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8


converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_model_quantized = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\Norbert\AppData\Local\Temp\tmpit4iuack\assets


INFO:tensorflow:Assets written to: C:\Users\Norbert\AppData\Local\Temp\tmpit4iuack\assets
c:\Users\Norbert\miniconda3\envs\dsss\lib\site-packages\tensorflow\lite\python\convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


In [12]:
with open("model_quantized.tflite", "wb") as f:
    f.write(tflite_model_quantized)

In [13]:
interpreter_quant = tf.lite.Interpreter(model_path="model_quantized.tflite")
interpreter_quant.allocate_tensors()

In [14]:
# Get input tensors
input_details_quant = interpreter_quant.get_input_details()
input_details_quant

[{'name': 'serving_default_conv2d_input:0',
  'index': 0,
  'shape': array([ 1, 28, 28,  1]),
  'shape_signature': array([-1, 28, 28,  1]),
  'dtype': numpy.int8,
  'quantization': (0.003921568859368563, -128),
  'quantization_parameters': {'scales': array([0.00392157], dtype=float32),
   'zero_points': array([-128]),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [15]:
# Get output tensors
output_details_quant = interpreter_quant.get_output_details()
output_details_quant

[{'name': 'StatefulPartitionedCall:0',
  'index': 20,
  'shape': array([ 1, 10]),
  'shape_signature': array([-1, 10]),
  'dtype': numpy.int8,
  'quantization': (0.00390625, -128),
  'quantization_parameters': {'scales': array([0.00390625], dtype=float32),
   'zero_points': array([-128]),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [20]:
# Randomly select 5 test images
random_indices = np.random.choice(range(len(x_test)), 5)

for i in random_indices:
    # Extract the single-channel image
    image = x_test[i]

    # Ensure the image is in the correct format for saving
    #if image.ndim == 3 and image.shape[-1] == 1:  # If single-channel with extra dimension
     #   image = image.squeeze()  # Remove the last dimension
    
    # Save the image with the class label in the filename
    cv2.imwrite(f"test_image_{y_test[i][0]}.png", image)